## NYC complaints 311 Calls: zipcodes

In this notebook we will exploit Pandas to perform data analysis on a dataset of calls to 311 (municipal calls, not emergency) in the New York City area.

Download the data from [here](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9) (Go to Export -> CSV). (**WARNING: > 16 GB of data**)

For this notebook, use on a smaller version of the data [here](https://drive.google.com/file/d/1EHYsxnN18LAKIPpZbtqjqCLi5hokG1ag/view?usp=sharing).

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

plt.rcParams['figure.figsize'] = (15, 5)
complaints = pd.read_csv('311_small.csv')

/var/folders/b3/cjvms6c10k77tgv6k_rw3jbh0000gn/T/ipykernel_14105/1126259180.py:8: DtypeWarning: Columns (8,31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints = pd.read_csv('311_small.csv')


In [2]:
complaints.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

#### drop columns 

In [3]:
complaints.shape

(999999, 41)

In [4]:
columns_to_drop = ['Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Descriptor', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment']
df = complaints.drop(columns=columns_to_drop)

In [5]:
df.shape

(999999, 20)

In [6]:
df['Incident Zip'].dtype

dtype('O')

In [7]:
df['Incident Zip'].isna().sum()

88509

Let's reduce this number.

# Part 1: basic conversions

In [8]:
all_zips = df['Incident Zip']

In [9]:
def float_and_str(x):
    """
    make sure to have only floats and strings. 
    Note: type(np.nan) = float64
    """
    try:
        return float(x)
    except:
        return str(x)
zip1 = all_zips.apply(lambda x: float_and_str(x))

In [10]:
zip1.apply(type).unique()

array([<class 'float'>, <class 'str'>], dtype=object)

In [11]:
# check what strings look like
zip1[zip1.apply(lambda x: isinstance(x, str))]

883       37214-0065
40092         NEWARK
40145     10423-0935
63485     90054-0807
64033     18773-9640
76409     08690-1717
90617     11797-9004
146452    44087-2340
154503           UNK
159120             ?
164769    91716-0500
166800       NO IDEA
179235    30353-0942
179493             ?
209339       UNKNOWN
244742    30092-2670
253124             *
329115             *
379814       UNKNOWN
392099    53566-8019
419089    12551-0831
505758    17108-0988
508559    17108-0988
511704       UNKNOWN
549824    85251-3643
606653    11590-5027
654119    11735-3946
682171    11802-9060
685967             *
688809    61702-3517
760548    55164-0437
924175    61702-3517
957902       UNKNOWN
983209    14225-1032
Name: Incident Zip, dtype: object

### Steps
  1. clean char strings to np.nan
  2. clean numeric string:
     * split on '-'
     * check against [ZIP CODES](https://bklyndesigns.com/new-york-city-zip-code/)
  3. populate missing data

### 1. Clean char strings to np.nan

In [12]:
def clean_strings(x):
    """
    Convert strings to np.nan
    """
    p1 = re.compile(r'[A-Z]+')  # any word
    p2 = re.compile(r'[*]+')    # * literal
    p3 = re.compile(r'[?]+')    # ? literal
    if isinstance(x, str):
        for p in [p1, p2, p3]:
            if re.search(p, x):
                return np.nan
    return x

In [13]:
zip2 = zip1.apply(lambda x: clean_strings(x))

In [14]:
zip2[zip2.apply(lambda x: isinstance(x, str))]

883       37214-0065
40145     10423-0935
63485     90054-0807
64033     18773-9640
76409     08690-1717
90617     11797-9004
146452    44087-2340
164769    91716-0500
179235    30353-0942
244742    30092-2670
392099    53566-8019
419089    12551-0831
505758    17108-0988
508559    17108-0988
549824    85251-3643
606653    11590-5027
654119    11735-3946
682171    11802-9060
688809    61702-3517
760548    55164-0437
924175    61702-3517
983209    14225-1032
Name: Incident Zip, dtype: object

### Clean  numeric strings
### 1. split on '-'

NYC Zip codes are 5 digits only

In [15]:
def split_on_dash(x):
    p_code = re.compile(r'^\d{5}')   # find a number 
    p = re.compile(r'-')
    try:
        if re.match(p_code, x) and re.search(p, x):
            tmp = x.split('-')[0]
            return float(tmp)
    except TypeError:
        return x

In [16]:
zip3 = zip2.apply(lambda x: split_on_dash(x))

In [17]:
zip3[zip3.apply(lambda x: isinstance(x, str))]

Series([], Name: Incident Zip, dtype: float64)

### 2. check against [ZIP CODES](https://bklyndesigns.com/new-york-city-zip-code/)

In [18]:
complaints.iloc[76409, 16]

'HAMILTON SQUARE'

In [19]:
complaints.iloc[146452, 16]

'TWINSBURG'

In [20]:
complaints.iloc[179235, 16]

'ATLANTA'

All outside NYC area... some noise

  * Manhattan: 10001-10282
  * Staten Island: 10301-10314
  * Bronx: 10451-10475
  * Queens: 11004-11109, 11351-11697
  * Brooklyn: 11201-11256
  
#### Invalid if:
  * x < 10001
  * 10282 < x < 10301
  * 10314 < x < 10451
  * 10475 < x < 11004
  * 11109 < x < 11201
  * 11256 < x < 11351
  * x > 11697
  

In [21]:
def clean_zip_ranges(x):
    # type(x) == float
    try: 
        code = int(x)
    except ValueError: # np.nan
        return np.nan
    if code < 10001 or code > 11697:
        return np.nan
    elif 10282 < code < 10301:
        return np.nan
    elif 10475 < code < 11004:
        return np.nan
    elif 11109 < code < 11201:
        return np.nan
    elif 11256 < code < 11351:
        return np.nan
    else:
        return code

In [22]:
zip4 = zip3.apply(lambda x: clean_zip_ranges(x))

In [23]:
zip4.shape

(999999,)

In [24]:
zip4.isna().sum()

89524

Weren't we suppose to reduce that?

In [25]:
zip4.dtypes

dtype('float64')

In [26]:
df['Incident Zip'] = zip4

### 3. populate missing data

In [27]:
place_desc = ['Incident Zip', 'Borough', 'Street Name', 
              'Cross Street 1', 'Cross Street 2', 
              'Intersection Street 1', 'Intersection Street 2', 
              'Address Type', 'City', 'Landmark', 'Facility Type']

In [28]:
df['Borough'].value_counts()

BROOKLYN         322885
QUEENS           240326
BRONX            182361
MANHATTAN        181866
STATEN ISLAND     53961
Unspecified       18502
Name: Borough, dtype: int64

In [29]:
unspec_boro = df['Borough'] == 'Unspecified'
df.loc[unspec_boro,'Borough'] = np.nan

In [30]:
df[place_desc].head(3)

,Incident Zip,Borough,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,11226.0,BROOKLYN,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11211.0,BROOKLYN,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN


In [31]:
df.dropna(how='all', subset=place_desc, inplace=True)

In [32]:
df.shape

(985714, 20)

In [33]:
for col in place_desc:
    print(f'{col :<22} {df[col].isna().sum() / df.shape[0] :.2%}')

Incident Zip           7.63%
Borough                0.44%
Street Name            13.21%
Cross Street 1         46.00%
Cross Street 2         46.20%
Intersection Street 1  87.36%
Intersection Street 2  87.38%
Address Type           7.04%
City                   7.59%
Landmark               98.89%
Facility Type          71.68%


In [34]:
# some useful filters
is_brooklyn = df['Borough'] == 'BROOKLYN'
is_manhattan = df['Borough'] == 'MANHATTAN'
is_queens = df['Borough'] == 'QUEENS'
is_bronx = df['Borough'] == 'BRONX'
is_staten = df['Borough'] == 'STATEN ISLAND'

In [35]:
def inspect(borough_mask, group_list, target):
    """
    Return a dataframe of per borough groups of distinct `group_list`
    attributes counting the number of `target` per group.
    """
    tmp = df.loc[borough_mask, place_desc]
    tmp['count'] = tmp.groupby(group_list)[target].transform('count')
    tmp.sort_values(by='Street Name', inplace=True)
    return tmp

In [36]:
group = ['Incident Zip', 'Street Name', 'City']
target = 'Street Name'
inspect(is_brooklyn, group, target).head()

,Incident Zip,Borough,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,count
396356,NaN,BROOKLYN,.DEKALB AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455159,NaN,BROOKLYN,1 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551777,11232.0,BROOKLYN,1 AVENUE,40 STREET,41 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,Precinct,17.0
170575,11220.0,BROOKLYN,1 AVENUE,54 STREET,55 STREET,54 STREET,55 STREET,NaN,BROOKLYN,1 AVENUE,NaN,11.0
669247,11220.0,BROOKLYN,1 AVENUE,56 STREET,57 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,11.0


In [37]:
# clean r'^\.'
df[df['Street Name'].str.match(r'^\.', na=False)]['Street Name']

396356    .DEKALB AVE
615835              .
Name: Street Name, dtype: object

In [38]:
df.loc[615835, 'Street Name'] = np.nan
df.loc[396356, 'Street Name'] = 'DEKALB AVE'

In [39]:
def find_all_zips(borough_mask):
    streets = df.loc[borough_mask, 'Street Name'].unique()
    multiple_zips = {}
    for street in streets:
        mask = df['Street Name'] == street
        street_to_zips = df.loc[(borough_mask & mask), 'Incident Zip'].unique()
        # pick up only when 2 alternative: [zipcode np.nan]
        if (len(street_to_zips) == 2) and (np.isnan(street_to_zips.sum())):
            multiple_zips[street] = street_to_zips
    return multiple_zips

#### Brooklyn

In [40]:
all_zips = find_all_zips(is_brooklyn)
print(len(all_zips))

864


In [41]:
all(len(zips) == 2 for _, zips in all_zips.items())

True

In [42]:
for k, v in all_zips.items():
    print(k, v)
    print(np.argwhere(~np.isnan(v)))
    print(v[np.argwhere(~np.isnan(v))[0][0]])
    break

95 STREET [11209.    nan]
[[0]]
11209.0


In [43]:
brooklyn_zips = [(street, zips[np.argwhere(~np.isnan(zips))[0][0]]) 
                for street, zips in all_zips.items()]

In [44]:
brooklyn_zips[0]

('95 STREET', 11209.0)

In [45]:
for street, zipcode in brooklyn_zips:
    mask = df['Street Name'] == street
    df.loc[(is_brooklyn & mask), 'Incident Zip'] = zipcode

#### Manhattan

In [46]:
all_zips = find_all_zips(is_manhattan)
print(len(all_zips))

143


In [47]:
manhattan_zips = [(street, zips[np.argwhere(~np.isnan(zips))[0][0]]) 
                for street, zips in all_zips.items()]
for street, zipcode in manhattan_zips:
    mask = df['Street Name'] == street
    df.loc[(is_manhattan & mask), 'Incident Zip'] = zipcode

#### Bronx

In [48]:
all_zips = find_all_zips(is_bronx)
print(len(all_zips))

bronx_zips = [(street, zips[np.argwhere(~np.isnan(zips))[0][0]]) 
                for street, zips in all_zips.items()]

for street, zipcode in bronx_zips:
    mask = df['Street Name'] == street
    df.loc[(is_bronx & mask), 'Incident Zip'] = zipcode

628


#### Queens

In [49]:
all_zips = find_all_zips(is_queens)
print(len(all_zips))

queens_zips = [(street, zips[np.argwhere(~np.isnan(zips))[0][0]]) 
                for street, zips in all_zips.items()]

for street, zipcode in queens_zips:
    mask = df['Street Name'] == street
    df.loc[(is_queens & mask), 'Incident Zip'] = zipcode

721


#### Staten Island

In [50]:
all_zips = find_all_zips(is_staten)
print(len(all_zips))

staten_zips = [(street, zips[np.argwhere(~np.isnan(zips))[0][0]]) 
                for street, zips in all_zips.items()]

for street, zipcode in staten_zips:
    mask = df['Street Name'] == street
    df.loc[(is_staten & mask), 'Incident Zip'] = zipcode

880


## After the first pass:

In [51]:
df.dropna(how='all', subset=place_desc, inplace=True)
df.shape

(985714, 20)

In [52]:
df['Incident Zip'].isna().sum()

61237

Better...
However this method has serious drawbacks in terms of performances.

### Deeper

In [53]:
df[['Borough', 'Cross Street 1']].drop_duplicates(keep=False).sort_values(by='Cross Street 1')

,Borough,Cross Street 1
341580,NaN,& IDEL PLACE
672342,BROOKLYN,'
241219,QUEENS,*146 TERRACE
885614,NaN,*149 RD
689786,BROOKLYN,*AVENUE H
...,...,...
925142,BRONX,YZNAGA PL
389078,STATEN ISLAND,ZEBRA PL
174616,QUEENS,ZION ST
749632,STATEN ISLAND,ZWICKY AVENUE


In [54]:
# remove r'^&' and r'^*' and r"^'"
df[df['Cross Street 1'].str.match(r'^\&', na=False)]['Cross Street 1']

341580    & IDEL PLACE
Name: Cross Street 1, dtype: object

In [55]:
df.loc[341580, 'Cross Street 1'] = 'IDEL PLACE'

In [56]:
df[df['Cross Street 1'].str.match(r'^\*', na=False)]['Cross Street 1']

226074     *PENNYFIELD AVE
241219        *146 TERRACE
628795    *GREENWAY AVENUE
689786           *AVENUE H
885614             *149 RD
Name: Cross Street 1, dtype: object

In [57]:
df.loc[226074, 'Cross Street 1'] = 'PENNYFIELD AVE'
df.loc[241219, 'Cross Street 1'] = '146 TERRACE'
df.loc[628795, 'Cross Street 1'] = 'GREENWAY AVENUE'
df.loc[689786, 'Cross Street 1'] = 'AVENUE H'
df.loc[885614, 'Cross Street 1'] = '149 RD'

In [58]:
df[df['Cross Street 1'].str.match(r"^\'", na=False)]['Cross Street 1']

672342    '
Name: Cross Street 1, dtype: object

In [59]:
df.loc[672342, 'Cross Street 1'] = np.nan

### About duplicates

In [60]:
df[['Borough', 'Cross Street 1']].drop_duplicates(keep=False).sort_values(by='Cross Street 1')

,Borough,Cross Street 1
185739,BRONX,1 AVENUE
974164,BROOKLYN,1 AVENUE & 2 AVENUE
727029,MANHATTAN,1 AVENUE LOOP
785902,STATEN ISLAND,1 BAY STREET
779933,BROOKLYN,1 CT
...,...,...
925142,BRONX,YZNAGA PL
389078,STATEN ISLAND,ZEBRA PL
174616,QUEENS,ZION ST
749632,STATEN ISLAND,ZWICKY AVENUE


In [61]:
df[['Borough', 'Cross Street 1', 'Cross Street 2']].drop_duplicates(keep=False).sort_values(by='Cross Street 2')

,Borough,Cross Street 1,Cross Street 2
387285,BROOKLYN,49 ST,1 AVE
92729,MANHATTAN,E 22 ST,1 AVE
704807,MANHATTAN,N D PERLMAN PL,1 AVE
519739,MANHATTAN,14 ST E,1 AVE
946967,MANHATTAN,10 ST E,1 AVE
...,...,...,...
996246,QUEENS,EDGEWATER ROAD,NaN
996796,BROOKLYN,WYCKOFF AVENUE,NaN
997323,BROOKLYN,PARK CIRCLE,NaN
997657,QUEENS,147 ST,NaN


Lots of duplicates on subset.

In [83]:
df['Cross Street 1'].isna().sum()

453411

In [84]:
# current situation
df['Incident Zip'].isna().sum()

61237

`df.duplicated` is your friend. 

In [70]:
columns_of_interest = ['Borough', 'Street Name', 'Cross Street 1']

# all duplicates in the columns_of_interest
dups = df[df.duplicated(columns_of_interest, keep=False)]

# groupby and find all unique Incident Zips
grouped = dups.groupby(columns_of_interest)['Incident Zip'].unique()

In [71]:
grouped

Borough        Street Name    Cross Street 1     
BRONX          138 ST E       138 ST E                   [nan]
               141 ST E       BROOK AVE                  [nan]
               144 ST E       BROOK AVE                  [nan]
               145 ST E       BROOK AVE                  [nan]
               148 ST E       BERGEN AVE                 [nan]
                                                       ...    
STATEN ISLAND  YUKON AVENUE   LOTT LANE              [10314.0]
               ZACHARY COURT  JEWETT AVENUE          [10310.0]
               ZOE STREET     DONGAN HILLS AVENUE    [10305.0]
                              LIBERTY AVENUE         [10305.0]
                              SEAVER AVENUE          [10305.0]
Name: Incident Zip, Length: 63078, dtype: object

In [72]:
# find all grouped with length == 2 and one of them is np.nan
print('before', grouped.size)
grouped = grouped[grouped.apply(lambda x: len(x) == 2 
                                and np.isnan(np.sum(x)))]
print('after', grouped.size)

before 63078
after 462


In [73]:
# transform series from narray to scalar
grouped = grouped.apply(lambda x: x[np.where(~np.isnan(x))[0][0]])

In [74]:
grouped[:4]

Borough  Street Name      Cross Street 1 
BRONX    ARNOW AVENUE     TIEMANN AVENUE     10469.0
         BATHGATE AVENUE  EAST 176 STREET    10457.0
                          EAST 187 STREET    10458.0
         BROADWAY         WEST 262 STREET    10471.0
Name: Incident Zip, dtype: float64

In [85]:
# find the indices (on dups) of the multi_index (in grouped)
# modify those indices on df
for index, value in grouped.items():
    indices = dups[(dups['Borough'] == index[0]) & 
             (dups['Street Name'] == index[1]) & 
             (dups['Cross Street 1'] == index[2])].index
    
    df.loc[indices,'Incident Zip'] = value

In [86]:
df['Incident Zip'].isna().sum()

60749

Let's do the same for 'Cross Street 2'

In [90]:
columns_of_interest = ['Borough', 'Street Name', 'Cross Street 2']
dups = df[df.duplicated(columns_of_interest, keep=False)]
grouped = dups.groupby(columns_of_interest)['Incident Zip'].unique()
grouped = grouped[grouped.apply(lambda x: len(x) == 2 
                                and np.isnan(np.sum(x)))]
grouped = grouped.apply(lambda x: x[np.where(~np.isnan(x))[0][0]])
len(grouped)

58

In [91]:
for index, value in grouped.items():
    indices = dups[(dups['Borough'] == index[0]) & 
             (dups['Street Name'] == index[1]) & 
             (dups['Cross Street 2'] == index[2])].index
    
    df.loc[indices,'Incident Zip'] = value

In [92]:
df['Incident Zip'].isna().sum()

60682

### Even deeper....

Only for Manhattan and 1st Avenue, as an example

In [93]:
m1 = complaints['Incident Zip'].isna()
m2 = complaints['Borough'].isna()
m3 = complaints['Street Name'].isna()
m4 = complaints['Cross Street 1'].isna()
m5 = complaints['Cross Street 2'].isna()
Ave1 = (complaints['Cross Street 1'] == '1 AVE') | (complaints['Intersection Street 1'] == '1 AVE')

In [94]:
complaints.loc[is_manhattan & (m1 & ~m2) & Ave1, :][place_desc].sort_values(by='Intersection Street 1')

,Incident Zip,Borough,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
4684,NaN,MANHATTAN,NaN,NaN,NaN,1 AVE,91 ST E,INTERSECTION,NaN,NaN,NaN
562892,NaN,MANHATTAN,NaN,1 AVE,54 ST E,1 AVE,54 ST E,INTERSECTION,NaN,NaN,NaN
572694,NaN,MANHATTAN,NaN,1 AVE,57 ST E,1 AVE,57 ST E,INTERSECTION,NaN,NaN,NaN
572851,NaN,MANHATTAN,NaN,1 AVE,39 ST E,1 AVE,39 ST E,INTERSECTION,NaN,NaN,NaN
573631,NaN,MANHATTAN,NaN,1 AVE,51 ST E,1 AVE,51 ST E,INTERSECTION,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
665575,NaN,MANHATTAN,ST CATHERINES PARK,1 AVE,2 AVE,NaN,NaN,BLOCKFACE,NaN,NaN,NaN
670619,NaN,MANHATTAN,FRST PARK,1 AVE,1 ST E,NaN,NaN,BLOCKFACE,NaN,NaN,NaN
773364,NaN,MANHATTAN,RALPH BUNCHE PARK,1 AVE,42 ST E,NaN,NaN,BLOCKFACE,NaN,NaN,NaN
873324,NaN,MANHATTAN,HAMMARSKJOLD PLAZA,1 AVE,2 AVE,NaN,NaN,BLOCKFACE,NaN,NaN,NaN
